## Baseline Machine Translation(OpenNMT)

<img src="https://www.nonprofitmarketingguide.com/blog/wp-content/uploads/2016/02/Jack-Sparrow-Deadlines.jpg" href="http://www.nonprofitmarketingguide.com/blog/2016/02/22/nonprofit-communications-confidential-deadlines/">
<p>Дэдлайны не страшны, когда есть решение из коробки для бейзлайна! :)</p>
<p>Сегодня мы будем использовать Google Colaboratory, поэтому часть кода нужна лишь для него.</p>

In [1]:
!pip3 install OpenNMT-tf

In [2]:
!pip install PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
fileId = drive.CreateFile({'id': 'Here_could_be_your_id!'})
fileId.GetContentFile('translation_dataset.zip')
!unzip translation_dataset.zip -d ./

Archive:  translation_dataset.zip
replace ./translation_dataset/train_en? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: ./translation_dataset/train_en  
  inflating: ./translation_dataset/val_en  
  inflating: ./translation_dataset/train_ru  
  inflating: ./translation_dataset/val_ru  
  inflating: ./translation_dataset/test_en  
  inflating: ./translation_dataset/test_ru  


In [0]:
# import os
# os.chdir('translation_dataset')

In [9]:
!ls

test_en  test_ru  train_en  train_ru  val_en  val_ru


In [12]:
!pwd train_en.txt

/content/translation_dataset


In [0]:
! onmt-build-vocab --size 50000 --save_vocab src-vocab.txt ./translation_dataset/train_en
! onmt-build-vocab --size 50000 --save_vocab tgt-vocab.txt ./translation_dataset/train_ru

In [0]:
# inspired by https://github.com/OpenNMT/OpenNMT-tf/blob/master/scripts/wmt/config/wmt_ende.yml
s = """model_dir: run/

data:
  train_features_file: ./translation_dataset/train_en
  train_labels_file: ./translation_dataset/train_ru
  eval_features_file: ./translation_dataset/val_en
  eval_labels_file: ./translation_dataset/val_ru
  source_words_vocabulary: src-vocab.txt
  target_words_vocabulary: tgt-vocab.txt
  
train:
  save_checkpoints_steps: 1000

eval:
  eval_delay: 1000
  external_evaluators: BLEU

infer:
  batch_size: 32
"""

In [0]:
with open('data.yml', 'w') as f:
    f.write(s)

In [0]:
! onmt-main train_and_eval --model_type NMTSmall --auto_config --config data.yml

INFO:tensorflow:Using parameters: {
  "data": {
    "eval_features_file": "./translation_dataset/val_en",
    "eval_labels_file": "./translation_dataset/val_ru",
    "source_words_vocabulary": "src-vocab.txt",
    "target_words_vocabulary": "tgt-vocab.txt",
    "train_features_file": "./translation_dataset/train_en",
    "train_labels_file": "./translation_dataset/train_ru"
  },
  "eval": {
    "batch_size": 32,
    "eval_delay": 1000,
    "exporters": "last",
    "external_evaluators": "BLEU"
  },
  "infer": {
    "batch_size": 32
  },
  "model_dir": "run/",
  "params": {
    "beam_width": 4,
    "clip_gradients": 5.0,
    "learning_rate": 0.0002,
    "length_penalty": 0.6,
    "optimizer": "AdamOptimizer",
    "param_init": 0.1
  },
  "score": {
    "batch_size": 64
  },
  "train": {
    "batch_size": 64,
    "batch_type": "examples",
    "bucket_width": 1,
    "maximum_features_length": 80,
    "maximum_labels_length": 80,
    "sample_buffer_size": -1,
    "save_checkpoints_steps": 